Ce notebook permet de récuperer un dataset stocké sur Google Drive, séparer les morceaux en Mineur de ceux en Majeur et les rescale en Do.
Deux fichiers en sorties : parts_major.zip, parts_minor.zip

In [ ]:
! pip install muspy

In [ ]:
import muspy as mp
import music21
import os

import random
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import shutil
import pretty_midi
import glob
from progressbar import ProgressBar
from google.colab import files

## A modifier pour changer le dataset à importer

In [ ]:
# get dataset from drive

! unzip "/content/drive/MyDrive/adl-piano-midi.zip" -d "data"

source_dir = '/content/data/adl-piano-midi/Jazz/'
target_dir = '/content/data/jazz_midi'

In [ ]:
# list to store .mid files
midi_files = []
# os.walk() returns subdirectories, file from current directory and 
# And follow next directory from subdirectory list recursively until last directory
for root, dirs, files in os.walk(r""+source_dir):
    for file in files:
        if file.endswith(".mid"):
            midi_files.append(os.path.join(root, file))
print("midi_files:",midi_files)

# check if exist or create target 
if not Path.exists(Path(target_dir)):
  os.mkdir(target_dir)

for file in midi_files:
  if not (Path('/content/data/jazz_midi/' + Path(file).name)).exists():
    shutil.move(file, '/content/data/jazz_midi')

_, _, files = next(os.walk(r""+target_dir))
file_count = len(files)
print("{}: {} files".format(target_dir, file_count))

In [ ]:
# Chemin d'accès au dossier d'entrée contenant les fichiers MIDI
input_folder = "/content/data/jazz_midi/"

# Liste des fichiers transposés
!rm -rf "/content/major_parts/"
!mkdir "/content/major_parts/"
!rm -rf "/content/minor_parts/"
!mkdir "/content/minor_parts/"
output_folder_major = "/content/major_parts/"
output_folder_minor = "/content/minor_parts/"

pbar = ProgressBar()

# Boucle à travers les 20 premiers fichier MIDI dans le dossier d'entrée
files = [f for f in os.listdir(input_folder) if f.endswith('.mid') or f.endswith('.midi')] #228 => crash

del files[228] #elem at index 228 causing crashes

for music_file in pbar(files):
    # Charger le fichier MIDI
    midi_file = music21.converter.parse(os.path.join(input_folder, music_file))

    # Vérifier si la musique est en majeur
    if midi_file.analyze('key').mode == 'major':
        # Déterminer la différence entre la tonalité actuelle et Do majeur
        intervalC_maj = music21.interval.Interval(midi_file.analyze('key').tonic, music21.pitch.Pitch('C'))
        # Transposer la musique dans la gamme de Do majeur
        transposed_midi_file = midi_file.transpose(intervalC_maj)

        new_file = os.path.join(output_folder_major, music_file[:-4] + '_transpose.mid') # ajouter '_transpose' au nom du fichier
        transposed_midi_file.write('midi', new_file)
    # Ou en mineur
    else:
        # Déterminer la différence entre la tonalité actuelle et Do mineur
        intervalC_min = music21.interval.Interval(midi_file.analyze('key').tonic, music21.pitch.Pitch('C'))
        # Transposer la musique dans la gamme de Do majeur
        transposed_midi_file = midi_file.transpose(intervalC_min)

        new_file = os.path.join(output_folder_minor, music_file[:-4] + '_transpose.mid') # ajouter '_transpose' au nom du fichier
        transposed_midi_file.write('midi', new_file)

In [ ]:
!zip -r /content/major_parts.zip /content/major_parts
!zip -r /content/minor_parts.zip /content/minor_parts

In [ ]:
files.download('/content/major_parts.zip')
files.download('/content/minor_parts.zip')